In [4]:
import modulos as md
import numpy as np
import pandas as pd
import pandas_ta as pta

'''
symbol='SUIUSDT'
lista=md.lista_de_monedas ()

for symbol in lista:
    try:
        data = md.obtiene_historial(symbol)
        data = md.estrategia_bb(data)
        resultado = md.backtesting(data)
        if float(resultado[4]) > 1015:
            print(f"{symbol} - {resultado[4]}")
    except:
        pass        
'''

import smtplib
import pandas as pd
import requests
from bs4 import BeautifulSoup
from price_parser import Price
import modulos as md
import constantes as cons


PRODUCT_URL_CSV = cons.pathroot+"products.csv"
SAVE_TO_CSV = True
PRICES_CSV = "prices.csv"
SEND_MAIL = True

def get_urls(csv_file):
    df = pd.read_csv(csv_file)
    return df

def get_response(url):
    response = requests.get(url)
    return response.text

def get_price(html):
    soup = BeautifulSoup(html, "lxml")
    el = soup.select_one(".price_color")
    price = Price.fromstring(el.text)
    return price.amount_float

def process_products(df):
    updated_products = []
    for product in df.to_dict("records"):
        html = get_response(product["url"])
        product["price"] = get_price(html)
        product["alert"] = product["price"] < product["alert_price"]
        updated_products.append(product)
    return pd.DataFrame(updated_products)

def get_mail(df):
    subject = "Price Drop Alert"
    body = df[df["alert"]].to_string()
    subject_and_message = f"Subject:{subject}\n\n{body}"
    return subject_and_message

def send_mail(df):
    message_text = get_mail(df)
    with smtplib.SMTP("smtp.server.address", 587) as smtp:
        smtp.starttls()
        md.sound()
        #smtp.login(mail_user, mail_pass)
        #smtp.sendmail(mail_user, mail_to, message_text)

def main():
    df = get_urls(PRODUCT_URL_CSV)
    print(df)
    df_updated = process_products(df)
    if SAVE_TO_CSV:
        df_updated.to_csv(PRICES_CSV, index=False, mode="a")
    if SEND_MAIL:
        send_mail(df_updated)

main()


Falla leyendo...


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:274: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000, commission=.002, exclusive_orders=True)


CRVUSDT - 1026.891655089706

Salida solicitada. 
